In [28]:
import duckdb
import pandas as pd
import numpy as np
import json

In [43]:
db = duckdb.connect('data/ufcstats.db')
fight_df = db.execute('SELECT * FROM raw_ufcstatsfighttable').fetchdf()
fighter_df = db.execute('SELECT * FROM raw_ufcstatsfightertable').fetchdf()
fight_df

,eventname,eventdate,eventlocation,id,weightclass,titlefight,method,round,time,timeformat,...,perf,sub,fight,ko,judge1,judge1_score,judge2,judge2_score,judge3,judge3_score
0,UFC Fight Night: Santos vs. Ankalaev,"March 12, 2022","Las Vegas, Nevada, USA",f10c2c84652ef573,Light Heavyweight Bout,---,KO/TKO,3,0:44,3 Rnd (5-5-5),...,http://1e49bc5171d173577ecd-1323f4090557a33db0...,---,---,---,---,---,---,---,---,---
1,UFC Fight Night: Santos vs. Ankalaev,"March 12, 2022","Las Vegas, Nevada, USA",b400c29fbeee0206,Bantamweight Bout,---,KO/TKO,1,2:07,3 Rnd (5-5-5),...,---,---,---,---,---,---,---,---,---,---
2,UFC Fight Night: Santos vs. Ankalaev,"March 12, 2022","Las Vegas, Nevada, USA",9cb78122a23e6a7d,Middleweight Bout,---,Submission,1,3:41,3 Rnd (5-5-5),...,http://1e49bc5171d173577ecd-1323f4090557a33db0...,---,---,---,---,---,---,---,---,---
3,UFC Fight Night: Santos vs. Ankalaev,"March 12, 2022","Las Vegas, Nevada, USA",b55de7d791b9f62d,Womens Flyweight Bout,---,Submission,2,2:15,3 Rnd (5-5-5),...,---,---,---,---,---,---,---,---,---,---
4,UFC Fight Night: Santos vs. Ankalaev,"March 12, 2022","Las Vegas, Nevada, USA",b70313eae7c24b27,Bantamweight Bout,---,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),...,---,---,---,---,Sal Damato,27 - 30.,Dave Hagen,27 - 30.,Mike Bell,28 - 29.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7151,UFC Fight Night: Cejudo vs. Dillashaw,"January 19, 2019","Brooklyn, New York, USA",7ddd21d48e3b34b9,Womens Flyweight Bout,---,Submission,2,1:50,3 Rnd (5-5-5),...,---,---,---,---,---,---,---,---,---,---
7152,UFC Fight Night: Cejudo vs. Dillashaw,"January 19, 2019","Brooklyn, New York, USA",4738cd4cd8d33eeb,Flyweight Bout,---,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),...,---,---,---,---,Doug Crosby,28 - 29.,Darryl Porter,28 - 29.,Tim Corrado,28 - 29.
7153,UFC Fight Night: Cejudo vs. Dillashaw,"January 19, 2019","Brooklyn, New York, USA",0261adcfc6e49150,Lightweight Bout,---,KO/TKO,2,4:59,3 Rnd (5-5-5),...,---,---,---,---,---,---,---,---,---,---
7154,UFC Fight Night: Cejudo vs. Dillashaw,"January 19, 2019","Brooklyn, New York, USA",8da2c348ba6001a0,Heavyweight Bout,---,DQ,2,2:28,3 Rnd (5-5-5),...,---,---,---,---,---,---,---,---,---,---


In [42]:
db.execute(
    '''
    '''
).fetchdf()

,Count
0,2390


In [40]:
db.execute('select * from information_schema.columns').fetchdf()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,ufcstats,main,raw_ufcstatsfightertable,fightername,1,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ufcstats,main,raw_ufcstatsfightertable,fighternickname,2,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ufcstats,main,raw_ufcstatsfightertable,record,3,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ufcstats,main,raw_ufcstatsfightertable,height,4,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ufcstats,main,raw_ufcstatsfightertable,weight,5,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,ufcstats,main,ufcstatsfightview,fighternickname,2,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,ufcstats,main,ufcstatsfightview,"regexp_extract_all(record, '\d+')",3,NaN,YES,VARCHAR[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,ufcstats,main,ufcstatsfighttable,fightername,1,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,ufcstats,main,ufcstatsfighttable,fighternickname,2,NaN,YES,VARCHAR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
db.close()